# Old 

In [ ]:
from udus_dataproc import *
if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'

from scipy import signal as sig
import matplotlib.pyplot as plt


In [ ]:
sig

In [ ]:
subjinds = [4]
tasks = ['hold', 'rest']
medconds = ['off', 'on']

subjind = subjinds[0]
task = tasks[0]
medcond = medconds[0]
MEGsrc_roi = ['HirschPt2011']
roi = MEGsrc_roi[0]
fname = getRawname(subjind,medcond,task)
#fname = getSrcname(subjind,medcond,task,roi)



fname_full = os.path.join(data_dir,fname)
fname_full += '_noMEG_resample_raw.fif'

f = mne.io.read_raw_fif(fname_full, None)
ra = f 
raws = {fname:ra}


In [ ]:
import scaleogram as scg
from mne.time_frequency import tfr_morlet

In [ ]:
k = fname
t1,t2 = 0,6
ts,te = ra.time_as_index([t1, t2])
chname = 'LFPL01'
r = getData(k,[chname],ts,te,raws_ = raws)
chdata, chtimes = r

In [ ]:
maxscale = 800  
#maxscale = 400  
base = 5
minscale = 4  # ~ 54 Hz  (smaller minscale means higher max freq, minscale 1 means max freq = sampling freq)
N = 65
pw = 3.5  # higher values -- larger density of higher freq
xs = np.arange(1, N + 1) / N
s = np.exp(xs) 
s = np.power(xs, pw)
s -= np.min(s)
smax = np.max(s)
spec_cwtscales = minscale + ( s/smax ) * maxscale/base; ttl = 'log {} spacing of scales'.format(base);
fr = pywt.scale2frequency('cmor1-1',spec_cwtscales) * 256
print(spec_cwtscales,fr)
#freqi = np.where(np.logical_and(fr >= f1, fr <= f2) )[0] 
#print(     len(freqi ))

In [ ]:
spec_wavelet = 'cmor1-1.5'
spec_wavelet = 'cmor1-2'
sgrams = {}

In [ ]:
NFFT = 256
sampleFreq = 256
specgram_scaling = 'spectrum'  # power, not normalized by freq band (i.e. not denisty)
c_parameter = 20
spec_FToverlap = 0.75  # power, not normalized by freq band (i.e. not denisty)

ra = raws[k]

info = mne.create_info(
    ch_names=[chname],
    ch_types=['eeg'],
    sfreq=sampleFreq
)


event_id = 1
events = np.array([
    [0, t1,event_id]
])  
sc = 1000
custom_epochs = mne.EpochsArray(sc * chdata[None,:], info, events, t1, event_id)

In [ ]:
notch_freq = 50
notch_width = 2
chdata2 = mne.filter.notch_filter(chdata[0,:], sampleFreq, notch_freq, notch_widths= notch_width)


In [ ]:
chdata2

In [ ]:
noverlap = NFFT*0.9
nr = 12; hh = 4
fig,axs = plt.subplots(nrows = nr, ncols=1, figsize = (20,nr * hh))
axind = 0

ax = axs[axind]
axind += 1

wlet = 'cmor1-1'
rr = pywt.cwt(chdata[0,:], spec_cwtscales , wlet, sampling_period = 1/sampleFreq)
Sxx, freqs = rr 
freqs = freqs[::-1]
Sxx = np.abs( Sxx[::-1,:] )
bins = chtimes
print(freqs)

sgrams['cwt_{}'.format(wlet)] = freqs,bins,Sxx

mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
#norm = mpl.colors.Normalize(vmin=mn,vmax=mx); 

im = ax.pcolormesh(bins, freqs, Sxx, cmap='inferno',norm=norm)
ax.set_title('pywt {}'.format(wlet))
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

###################################

ax = axs[axind]
axind += 1

wlet = 'cmor2-1'
rr = pywt.cwt(chdata[0,:], spec_cwtscales , wlet, sampling_period = 1/sampleFreq)
Sxx, freqs = rr 
freqs = freqs[::-1]
Sxx = np.abs( Sxx[::-1,:] )
bins = chtimes
print(freqs)

sgrams['cwt_{}'.format(wlet)] = freqs,bins,Sxx

mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
#norm = mpl.colors.Normalize(vmin=mn,vmax=mx); 

im = ax.pcolormesh(bins, freqs, Sxx, cmap='inferno',norm=norm)
ax.set_title('pywt {}'.format(wlet))
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

#######################################
ax = axs[axind]
axind += 1

wlet = 'cmor1-2'
rr = pywt.cwt(chdata[0,:], spec_cwtscales , wlet, sampling_period = 1/sampleFreq)
Sxx, freqs = rr 
freqs = freqs[::-1]
Sxx = np.abs( Sxx[::-1,:] )
bins = chtimes
print(freqs)

sgrams['cwt_{}'.format(wlet)] = freqs,bins,Sxx

mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
#norm = mpl.colors.Normalize(vmin=mn,vmax=mx); 

im = ax.pcolormesh(bins, freqs, Sxx, cmap='inferno',norm=norm)
ax.set_title('pywt {}'.format(wlet))
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

#######################################
ax = axs[axind]
axind += 1
par = {'x':chdata, 'fs':sampleFreq, 'nperseg':NFFT, 'scaling':specgram_scaling, 'noverlap':noverlap }

par['c_parameter' ] = c_parameter
rr = spectrogram_lspopt(**par)
freqs,bins,Sxx = rr
sgrams['lspopt_{}'.format(par['c_parameter' ])] = rr
       
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, freqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title('lspopt c={}'.format(par['c_parameter' ]))
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

###########################

ax = axs[axind]
axind += 1
par['c_parameter' ] = 3
rr = spectrogram_lspopt(**par)
sgrams['lspopt_{}'.format(par['c_parameter' ])] = rr
freqs,bins,Sxx = rr
       
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, freqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title('lspopt c={}'.format(par['c_parameter' ]))
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

##############################

ax = axs[axind]
axind += 1
del par['c_parameter' ] 
rr = sig.spectrogram(**par)
sgrams['scipy'] = rr
freqs,bins,Sxx = rr
       
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, freqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title('scipy')
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

##############################

ax = axs[axind]
axind += 1
mnefreqs = np.arange(5., 50., 1.)
#freqs,bins, = rr
n_cycles = 3
Sxx = tfr_morlet(custom_epochs, freqs=mnefreqs,
                   n_cycles=n_cycles, return_itc=False)
Sxx = Sxx.data
sgram_name = 'mne_{}cycles'.format(n_cycles)
sgrams[sgram_name] = mnefreqs,chtimes,Sxx[0,:]
#pp = [0,:] #- np.mean( power.data )
#power.plot( baseline=(0., 0.1), mode='mean', vmin=vmin,vmax= vmax, 
#           axes=ax, show=False, colorbar=False)

#ax.pcolormesh(chtimes,mnefreqs, pp, 
#          norm= mpl.colors.LogNorm(vmin=np.min(power.data), vmax=np.max(power.data)))
    
bins = chtimes
    
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, mnefreqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title(sgram_name)
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

##############################

ax = axs[axind]
axind += 1
mnefreqs = np.arange(5., 50., 1.)
#freqs,bins, = rr
n_cycles = 7
Sxx = tfr_morlet(custom_epochs, freqs=mnefreqs,
                   n_cycles=n_cycles, return_itc=False)
Sxx = Sxx.data
sgram_name = 'mne_{}cycles'.format(n_cycles)
sgrams[sgram_name] = mnefreqs,chtimes,Sxx[0,:]
#pp = power.data[0,:] #- np.mean( power.data )
#power.plot( baseline=(0., 0.1), mode='mean', vmin=vmin,vmax= vmax, 
#           axes=ax, show=False, colorbar=False)

#ax.pcolormesh(chtimes,mnefreqs, pp, 
#          norm= mpl.colors.LogNorm(vmin=np.min(power.data), vmax=np.max(power.data)))
    
bins = chtimes
    
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, mnefreqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title(sgram_name)
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)

##############################

ra = raws[k]

info = mne.create_info(
    ch_names=[chname],
    ch_types=['eeg'],
    sfreq=sampleFreq
)

event_id = 1
events = np.array([
    [0, t1,event_id]
])  
sc = 1.
custom_epochs = mne.EpochsArray(sc * chdata[None,:], info, events, t1, event_id)

ax = axs[axind]
axind += 1
mnefreqs = np.arange(5., 50., 1.)
#freqs,bins, = rr
freq2cycles_coef = 1
n_cycles = mnefreqs / freq2cycles_coef
Sxx = tfr_morlet(custom_epochs, freqs=mnefreqs,
                   n_cycles=n_cycles, return_itc=False)
Sxx = Sxx.data
sgram_name = 'mne_freqscaled_{}'.format(freq2cycles_coef)
sgrams[sgram_name] = mnefreqs,chtimes,Sxx[0,:]
#pp = power.data[0,:] #- np.mean( power.data )
#power.plot( baseline=(0., 0.1), mode='mean', vmin=vmin,vmax= vmax, 
#           axes=ax, show=False, colorbar=False)

#ax.pcolormesh(chtimes,mnefreqs, pp, 
#          norm= mpl.colors.LogNorm(vmin=np.min(power.data), vmax=np.max(power.data)))
    
bins = chtimes
    
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, mnefreqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title(sgram_name)
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)


##########################################


ax = axs[axind]
axind += 1
mnefreqs = np.arange(5., 50., 1.)
#freqs,bins, = rr
freq2cycles_coef = 4
n_cycles = mnefreqs / freq2cycles_coef
Sxx = tfr_morlet(custom_epochs, freqs=mnefreqs,
                   n_cycles=n_cycles, return_itc=False)
Sxx = Sxx.data
sgram_name = 'mne_freqscaled_{}'.format(freq2cycles_coef)
sgrams[sgram_name] = mnefreqs,chtimes,Sxx[0,:]
#pp = power.data[0,:] #- np.mean( power.data )
#power.plot( baseline=(0., 0.1), mode='mean', vmin=vmin,vmax= vmax, 
#           axes=ax, show=False, colorbar=False)

#ax.pcolormesh(chtimes,mnefreqs, pp, 
#          norm= mpl.colors.LogNorm(vmin=np.min(power.data), vmax=np.max(power.data)))
    
bins = chtimes
    
mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
im = ax.pcolormesh(bins, mnefreqs, Sxx[0,:,:], cmap='inferno',norm=norm)
ax.set_title(sgram_name)
ax.set_ylim(0,50)
ax.set_xlim(t1,t2)




###########################################

ax = axs[axind]
axind += 1

#wlet = lambda M,s : sig.morlet(M*s, w=7.0 /  (s/10) , s=s, complete=True)
#wlet = lambda M,s : sig.morlet(M*s, w=7.0 /  (s/100) , s=s, complete=True)
wlet = lambda M,s : sig.morlet(M*s/2, w=7.0 *  (0.5*s/np.max(s)) , s=s, complete=True)

widths = np.arange(1, 60, 1) 
Sxx = sig.cwt(chdata[0,:], wlet, widths)
Sxx=np.abs(Sxx) ** 2

sgram_name = 'scipy.cwt'
sgrams[sgram_name] = mnefreqs,chtimes,Sxx

#t = np.linspace(-1, 1, 200, endpoint=False)
#sig  = np.cos(2 * np.pi * 7 * t) + signal.gausspulse(t - 0.4, fc=2)
#ax.pcolormesh(bins,freqs,cwtmatr, extent=[-1, 1, 31, 1], cmap='PRGn', aspect='auto',
#           vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())

mn = np.min(Sxx)
mx = np.max(Sxx)
norm = mpl.colors.LogNorm(vmin=mn,vmax=mx); 
ys  = 1/widths
ys = widths
im = ax.pcolormesh(bins, ys, Sxx[:,:], cmap='inferno',norm=norm)
ax.set_title(sgram_name)
ax.set_ylim(0,np.max(ys))
ax.set_xlim(t1,t2)
ax.invert_yaxis()

###########################################

ax = axs[axind]
axind += 1
scg.cws(chtimes, chdata[0,:], scales=spec_cwtscales, ylabel="Freq", xlabel='Time',ax=ax, cscale='log')
ax.set_xlim(t1,t2)
a = ax.get_yticks()
print(a)
aa = [float(tick) for tick in a]
freqs = np.array( [(1/ (tval+1e-5) ) for tval in aa] )
aaa = [ '{:.0f}'.format(f ) for f in freqs]
ax.set_yticklabels(aaa)
ax.set_title('scg.cws')

#sgrams['cwt_{}'.format(wlet)] = freqs,bins,Sxx

In [ ]:
f1,f2 = 15,35
nr = len(sgrams)
hh = 3
fig,axs = plt.subplots(nrows=nr, ncols=1, figsize= (20,nr*hh))

for axind, st in enumerate( sorted(sgrams.keys() ) ):
    #print(st)
    freqs,bins,Sxx = sgrams[st]
    freqi = np.where(np.logical_and(freqs >= f1, freqs <= f2) )[0]
    print(st,len(freqi))
    if isinstance(Sxx[0,0], complex):
        Sxx = np.square( np.abs(Sxx) )
        
    if st.find('cwt') >=0 or st.find('scg') >= 0 or st.find('mne') >= 0:
        tp  = 'scg'
    else:
        tp = 'specgram'
    if Sxx.ndim == 3 and Sxx.shape[0] == 1:
        Sxx = Sxx[0,:]
        
    bandpow = np.sum(Sxx[freqi,:], axis = 0)
    
    ax = axs[axind]
    ax.plot(bins,bandpow)
    ax.set_title(st)
    ax.set_xlim(t1,t2)
    if tp == 'scg':
        bini= np.where( np.logical_and(bins > t1 + 0.5 , bins < t2 - 0.5 ) )[0]
        mx = np.max( bandpow[bini] )
        ax.set_ylim(0,mx)
    


In [ ]:
plt.plot(t,sigg)

In [ ]:
fig,axs = plt.subplots(ncols=2,figsize=(14,4))
t = np.linspace(-1, 1, 200, endpoint=False)
sigg  = np.cos(2 * np.pi * 7 * t) + signal.gausspulse(t - 0.4, fc=2)
widths = np.arange(1, 31)
wlet = lambda M,s : sig.morlet(M, w=7.0, s=s/60, complete=True)
cwtmatr = signal.cwt(sigg, wlet, widths)
ax = axs[0]
ax.imshow(np.abs( cwtmatr), extent=[-1, 1, 31, 1], cmap='PRGn', aspect='auto',
           vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())

ax = axs[1]
cwtmatr = signal.cwt(sigg, sig.ricker, widths)
ax.imshow(np.abs( cwtmatr), extent=[-1, 1, 31, 1], cmap='PRGn', aspect='auto',
           vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())


In [ ]:
cwtmatr.shape